In [1]:
import os
import json
import pandas as pd
import traceback

In [6]:
from langchain_community.chat_models import ChatOpenAI



In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
KEY=os.getenv("OPENAI_API_KEY")

In [13]:
llm=ChatOpenAI(opneai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

C:\Users\saura\AppData\Local\Temp\ipykernel_17824\1581607659.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(opneai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)
WARNING! opneai_api_key is not default parameter.
                    opneai_api_key was transferred to model_kwargs.
                    Please confirm that opneai_api_key is what you intended.
